In [ ]:
import re

import matplotlib.pyplot as plt
from commonroad.common.util import Interval, AngleInterval
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.ConversionAnalyzer.EAnalyzer import EAnalyzer
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason
from OpenSCENARIO2CR.util.AbsRel import AbsRel

In [ ]:
scenario_path = ""
run_viewer = True
print_plots = False
plots_step = 5
plot_limit = 20  # If non-null the renderer follows the ego vehicle
following_obstacle_index = 0

In [ ]:
# Setup EsminiWrapper
esmini_wrapper = EsminiWrapperProvider(preferred_version="v2.26.5").provide_esmini_wrapper()
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False

In [ ]:
if run_viewer:
    wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
    wrapper.grace_time = None
    wrapper.max_time = 60.0
    wrapper.ignored_level = EStoryBoardElementType.ACT
    wrapper.view_scenario(scenario_path)

In [ ]:
# Setup Converter
converter = Osc2CrConverter(
    sim_wrapper=esmini_wrapper,
    delta_t=0.1,
    goal_state_time_step=AbsRel[Interval](Interval(-5, 0), is_absolute=False),
    goal_state_position_width=5,
    goal_state_position_length=25,
)
converter.goal_state_position_use_ego_rotation = True,
converter.goal_state_orientation = AbsRel[AngleInterval](AngleInterval(-0.8, 0.8), is_absolute=False),
converter.keep_ego_vehicle = True,
converter.trim_scenario = False,
converter.ego_filter = re.compile(r".*ego.*", re.IGNORECASE),
converter.use_implicit_odr_file = True,
converter.analyzers = [EAnalyzer.STL_MONITOR, EAnalyzer.DRIVABILITY_CHECKER]

result = converter.run_conversion(scenario_path)
if isinstance(result, EFailureReason):
    print(f"Scenario failed due to {result}")

In [ ]:
# plot every second time_stamp of the scenario
if print_plots and not isinstance(result, EFailureReason):
    scenario, planning_problem_set, statistics = result
    for i in range(0, max([obst.prediction.final_time_step for obst in scenario.dynamic_obstacles]) + 1, plots_step):
        plt.figure(figsize=(5, 5))

        if plot_limit is None:
            rnd = MPRenderer()
        else:
            rnd = MPRenderer(plot_limits=[-plot_limit, plot_limit, -plot_limit, plot_limit],
                             draw_params={
                                 "focus_obstacle_id": scenario.obstacles[following_obstacle_index].obstacle_id})
        # plot the scenario at different time step
        scenario.draw(rnd, draw_params={'time_begin': i})
        # plot the planning problem set
        planning_problem_set.draw(rnd)
        rnd.render()